In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, os, sys, shutil
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('./icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('./icr-identify-age-related-conditions/test.csv')
meta = pd.read_csv('./icr-identify-age-related-conditions/greeks.csv')

In [3]:
train_init = train.copy()
train['Alpha'] = meta['Alpha']
train['Beta'] = meta['Beta']
train['Gamma'] = meta['Gamma']
train['Delta'] = meta['Delta']

In [4]:
# 处理greeks
# Alpha为A的时候为0，其余为1
train['Alpha'] = train['Alpha'].apply(lambda x: 0 if x == 'A' else 1)

In [5]:
# 将数据集中唯一的离散特征转为01，这可能表明患者的性别
train['EJ'] = train['EJ'].replace({'A': 0, 'B': 1}).astype(float)
test['EJ']  = test['EJ'].replace({'A': 0, 'B': 1}).astype(float)
train_init['EJ'] = train_init['EJ'].replace({'A': 0, 'B': 1}).astype(float)
# 缺失值处理，用中位数填充
train['BQ'].fillna(0, inplace=True)
train.fillna(train.median(), inplace=True)
test.fillna(test.median(), inplace=True)
train_init.fillna(train_init.median(), inplace=True)
# 移除ID列
train_id = train['Id'].copy()
test_id  =  test['Id'].copy()
train = train.drop(['Id'], axis=1)
test  =  test.drop(['Id'], axis=1)

In [6]:
# 将gamma中的M和N都转为0，G，H，E，F，A，B的转为1
train['Gamma'] = train['Gamma'].replace({'M': 0, 'N': 0, 'G': 1, 'H': 1, 'E': 1, 'F': 1, 'A': 1, 'B': 1})
# 将Beta中的C0,B1,A2
train['Beta'] = train['Beta'].replace({'C': 0, 'B': 1, 'A': 2})
# 将Delta中的B0,A1,C1,D2
train['Delta'] = train['Delta'].replace({'B': 0, 'A': 1, 'C': 1, 'D': 2})

train.head()


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,72.611063,2003.810319,22.136229,69.834944,0.120343,1,1,0,1,2
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,72.611063,27981.562750,29.135430,32.131996,21.978000,0,0,0,0,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,88.609437,13676.957810,28.022851,35.192676,0.196941,0,0,0,0,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,82.416803,2094.262452,39.948656,90.493248,0.155829,0,0,0,0,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,146.109943,8524.370502,45.381316,36.262628,0.096614,1,1,1,1,0


In [7]:
# 连续变量归一化处理
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric_columns = [_ for _ in train.columns if _ not in ['EJ', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Class']]
train[numeric_columns] = scaler.fit_transform(train[numeric_columns])
test[numeric_columns] = scaler.transform(test[numeric_columns])
train.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta
0,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,-0.405383,...,-0.410260,-0.655511,-0.948991,0.531241,-0.814049,1,1,0,1,2
1,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,0.048541,...,-0.410260,0.687893,-0.238862,-0.509218,1.304860,0,0,0,0,0
2,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,-0.071089,...,-0.299210,-0.051850,-0.351743,-0.424754,-0.806623,0,0,0,0,0
3,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,-0.391109,...,-0.342195,-0.650833,0.858232,1.101332,-0.810609,0,0,0,0,0
4,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,0.125327,...,0.099920,-0.318309,1.409422,-0.395228,-0.816349,1,1,1,1,0


In [8]:
# 抛弃高度相关的特征
drop_cols = ['BZ','CL','EH','GL']  # 高度相关
drop_cols2 = ['DY','CB','GB','CH','DL','CU','FS','AZ','GE','EG','EP']  # 无用特征，同一点取得极值
drop_cols3 = ['BZ', 'DV', 'EH', 'FD ']
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

In [9]:
# ytrain为离散
ytrain = train[['Class', 'Alpha', 'Beta', 'Gamma', 'Delta']]
train.drop(['Class', 'Alpha', 'Beta', 'Gamma', 'Delta'], axis=1, inplace=True)

In [10]:
train.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI
0,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,-0.405383,...,-0.112922,-2.226608,0.162821,-0.035806,-0.250437,-0.940094,-0.410260,-0.655511,-0.948991,0.531241
1,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,0.048541,...,-0.311056,0.084542,-0.457925,-0.060566,0.113640,-1.145070,-0.410260,0.687893,-0.238862,-0.509218
2,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,-0.071089,...,-0.173811,0.517060,0.198663,-0.051023,0.597343,1.637944,-0.299210,-0.051850,-0.351743,-0.424754
3,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,-0.391109,...,0.058201,1.617160,0.060364,-0.060566,-0.104806,-0.219883,-0.342195,-0.650833,0.858232,1.101332
4,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,0.125327,...,0.520331,1.212850,0.237302,0.896815,-0.229632,-0.432313,0.099920,-0.318309,1.409422,-0.395228


# 模型搭建
先直接对Class预测

In [11]:
# 数据集分割
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
x_train, x_test, y_train, y_test = train_test_split(train, ytrain, test_size=0.2, random_state=42)
scale_pos_weight = 4.712962962962963
print(x_train.shape)
print(y_train.shape)

(493, 52)
(493, 5)


In [12]:
def ScoreMetric(ytrue, ypred):
    nc = np.bincount(ytrue);
    return log_loss(ytrue, ypred, sample_weight = 1 / nc[ytrue], eps=1e-15);

In [13]:
from catboost import CatBoost, CatBoostClassifier
cat = CatBoostClassifier(random_state=42)
cat.fit(x_train, y_train['Class'])
cat_pred = cat.predict(x_test)
y_pred = cat.predict_proba(x_test)
cat_accuracy = accuracy_score(cat_pred, y_test['Class'])

cat_accuracy

Learning rate set to 0.007617
0:	learn: 0.6862818	total: 144ms	remaining: 2m 24s
1:	learn: 0.6785904	total: 149ms	remaining: 1m 14s
2:	learn: 0.6711415	total: 153ms	remaining: 50.9s
3:	learn: 0.6631032	total: 158ms	remaining: 39.2s
4:	learn: 0.6558536	total: 162ms	remaining: 32.3s
5:	learn: 0.6495764	total: 166ms	remaining: 27.5s
6:	learn: 0.6427384	total: 171ms	remaining: 24.2s
7:	learn: 0.6339141	total: 176ms	remaining: 21.8s
8:	learn: 0.6256367	total: 180ms	remaining: 19.8s
9:	learn: 0.6197541	total: 185ms	remaining: 18.3s
10:	learn: 0.6138121	total: 191ms	remaining: 17.1s
11:	learn: 0.6048446	total: 196ms	remaining: 16.1s
12:	learn: 0.6002516	total: 200ms	remaining: 15.2s
13:	learn: 0.5940357	total: 205ms	remaining: 14.4s
14:	learn: 0.5875136	total: 210ms	remaining: 13.8s
15:	learn: 0.5818911	total: 214ms	remaining: 13.2s
16:	learn: 0.5753831	total: 219ms	remaining: 12.7s
17:	learn: 0.5679193	total: 225ms	remaining: 12.3s
18:	learn: 0.5608603	total: 231ms	remaining: 11.9s
19:	learn

0.967741935483871

In [14]:
p0 = y_pred[:,1]
# p0[p0 > 0.90] = 1
# p0[p0 < 0.10] = 0
ScoreMetric(y_test['Class'], p0)
# p0

0.3366599987328377

# 正式的KFold交叉验证 + Optuna模型调参

In [15]:
import optuna, tune

from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, log_loss
from sklearn.metrics import make_scorer, accuracy_score, log_loss
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit

# fold
from sklearn.model_selection import KFold, StratifiedKFold


In [16]:
def objective(trial):
    '''

    '''
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical( "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"] ),
        "used_ram_limit": "3gb",
        "eval_metric": "Accuracy",
        "random_state": 42,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    log_loss_scores = []
    # 划分验证集
    for train_index, val_index in skf.split(x_train, y_train['Class']):
        x_train_, x_val = x_train.iloc[train_index], x_train.iloc[val_index]
        y_train_, y_val = y_train['Class'].iloc[train_index], y_train['Class'].iloc[val_index]
        classifier = CatBoostClassifier(**param, verbose=0)
        classifier.fit(x_train_, y_train_, eval_set=[(x_val, y_val)], early_stopping_rounds=180, verbose=0)
        y_pred_proba = classifier.predict_proba(x_val)[:, 1]
        log_loss_scores.append(ScoreMetric(y_val, y_pred_proba))
    return np.mean(log_loss_scores)

if __name__ == "__main__":
    sampler = TPESampler(seed=42)
    pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
    study = optuna.create_study(pruner=pruner, direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=500)

    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-08-02 20:44:50,797] A new study created in memory with name: no-name-0bd3d1e8-9a3c-410c-b50f-9720738b69de
[I 2023-08-02 20:46:04,844] Trial 0 finished with value: 0.5259240659194301 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.05395030966670229, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.5105903209394755}. Best is trial 0 with value: 0.5259240659194301.
[I 2023-08-02 20:46:14,939] Trial 1 finished with value: 0.5292645099076044 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.06798962421591129, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.1955370866274525}. Best is trial 0 with value: 0.5259240659194301.
[I 2023-08-02 20:46:22,132] Trial 2 finished with value: 0.689652155918221 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.019594972058679168, 'depth': 5, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.5

KeyboardInterrupt: 

In [ ]:
model = CatBoostClassifier(**trial.params)
model.fit(x_train, y_train['Class'])
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)[:, 1]
ScoreMetric(y_test['Class'], y_pred_proba)
accuracy_score(y_pred, y_test['Class'])

0.9516129032258065

In [ ]:
plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 66,
                    219],
              'y': [0.5967335254562047, 0.6433822087003274, 0.6184598051405754,
                    0.5233455573713329, 0.4135077642757382, 0.6033821687360186,
                    0.38797169527703657, 0.48554161494647613, 0.6513827334700728,
                    0.43322869655213225, 0.5276990147146629, 0.3895251437757331,
                    0.40468399162037577, 0.3537327900333324, 0.389529660605126,
                    0.4147123753726313, 0.4157193493222701, 0.4648339002118201]},
             {'name': 'Best Value',
              'type': 'scatter',
              'x': [1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 66,
                    219],
              'y': [0.5967335254562047, 0.5967335254562047, 0.5967335254562047,
                    0.5233455573713329, 0.4135077642757382, 0.4135077642757382,
                    0.38797169527703657, 0.38797169527703657, 0.38797169527703657,
                    0.38797169527703657, 0.38797169527703657, 0.38797169527703657,
                    0.38797169527703657, 0.3537327900333324, 0.3537327900333324,
                    0.3537327900333324, 0.3537327900333324, 0.3537327900333324]}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

In [ ]:
plot_slice(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                                   17, 18, 19, 66, 219],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': True},
              'mode': 'markers',
              'showlegend': False,
              'type': 'scatter',
              'x': [0.574485163632042, 0.23713402899336497, 0.04264813784432918,
                    0.08032068562667222, 0.012197768563438372, 0.1821930437934653,
                    1.9488135833929595e-07, 7.593034903208851e-08,
                    1.033375989766645e-06, 0.0066946948903108105,
                    8.909025084911593e-06, 0.0005852079598515014,
                    0.0002219322141207024, 1.0601341392077246e-08,
                    1.2862740622795762e-08, 1.0456940308891758e-08,
                    3.4246884993023374e-07, 5.009324548277015e-06],
              'xaxis': 'x',
              'y': [0.5967335254562047, 0.6433822087003274, 0.6184598051405754,
                    0.5233455573713329, 0.4135077642757382, 0.6033821687360186,
                    0.38797169527703657, 0.48554161494647613, 0.6513827334700728,
                    0.43322869655213225, 0.5276990147146629, 0.3895251437757331,
                    0.40468399162037577, 0.3537327900333324, 0.389529660605126,
                    0.4147123753726313, 0.4157193493222701, 0.4648339002118201],
              'yaxis': 'y'},
             {'marker': {'color': [1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                                   17, 18, 19, 66, 219],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': False},
              'mode': 'markers',
              'showlegend': False,
              'type': 'scatter',
              'x': [gbtree, gblinear, dart, gbtree, gbtree, gblinear, gbtree,
                    dart, dart, dart, gbtree, gbtree, gbtree, gbtree, gbtree,
                    gbtree, gbtree, gbtree],
              'xaxis': 'x2',
              'y': [0.5967335254562047, 0.6433822087003274, 0.6184598051405754,
                    0.5233455573713329, 0.4135077642757382, 0.6033821687360186,
                    0.38797169527703657, 0.48554161494647613, 0.6513827334700728,
                    0.43322869655213225, 0.5276990147146629, 0.3895251437757331,
                    0.40468399162037577, 0.3537327900333324, 0.389529660605126,
                    0.4147123753726313, 0.4157193493222701, 0.4648339002118201],
              'yaxis': 'y2'},
             {'marker': {'color': [1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                                   17, 18, 19, 6

In [ ]:
plot_param_importances(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [lambda (FloatDistribution):
                                0.0010211392978202668<extra></extra>, booster
                                (CategoricalDistribution):
                                0.04281909881010983<extra></extra>, subsample
                                (FloatDistribution):
                                0.07541359284822767<extra></extra>, alpha
                                (FloatDistribution):
                                0.15290526812621347<extra></extra>,
                                colsample_bytree (FloatDistribution):
                                0.7278409009176288<extra></extra>],
              'marker': {'color': 'rgb(66,146,198)'},
              'orientation': 'h',
              'text': [<0.01, 0.04, 0.08, 0.15, 0.73],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.0010211392978202668, 0.04281909881010983,
                    0.07541359284822767, 0.15290526812621347, 0.7278409009176288],
              'y': [lambda, booster, subsample, alpha, colsample_bytree]}],
    'layout': {'showlegend': False,
               'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Importance for Objective Value'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})